In [1]:
import tensorflow as tf
import os
import IPython.display as ipd

2024-05-16 10:32:46.214014: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 10:32:46.707929: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# alphanumeric order
labels = sorted(os.listdir("./../../speech_commands_v0.02"))

idx_label_dict = {label: idx for idx, label in enumerate(labels)}
    
up_label = idx_label_dict["up"]
down_label = idx_label_dict["down"]
right_label = idx_label_dict["right"]
left_label = idx_label_dict["left"]

FileNotFoundError: [Errno 2] No such file or directory: './speech_commands_v0.02'

In [ ]:
train_ds, test_ds = tf.keras.utils.audio_dataset_from_directory(
                            directory="./speech_commands_v0.02",
                            batch_size=None,
                            validation_split=0.2,
                            seed=0,
                            output_sequence_length=16000,
                            subset='both'
                        )

In [ ]:
def relabel(label):

    if label == up_label:
        return 0
    elif label == right_label:
        return 1
    elif label == down_label:
        return 2
    elif label == left_label:
        return 3
    else:
        return 4

def prepare_data(dataset):

    dataset = dataset.map(lambda wav, label: (wav, relabel(label)))

    # Flatten: (num_samples, 1) -> (num_samples)
    dataset = dataset.map(lambda wav, label: (tf.reshape(wav, (-1,)), label))

    # # Spectrogram (height, width, 1)
    # dataset = dataset.map(lambda wav, label:(get_spectrogram(wav), label))

    # # One hot target
    # dataset = dataset.map(lambda spectrogram, label: (spectrogram, tf.one_hot(label, depth=8)))

    # # Cache
    # dataset = dataset.cache()
    
    # #
    # # Shuffle, batch, prefetch
    # #
    # dataset = dataset.shuffle(1000)
    # dataset = dataset.batch(BATCH_SIZE)
    # dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset

In [ ]:
train_ds = train_ds.apply(prepare_data)

In [ ]:
for wav, label in train_ds.take(50):
    print(label)
    ipd.display(ipd.Audio(wav, rate=16000))